In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
import statsmodels.api as sm

import requests

In [2]:
dfPlaylists = pd.read_csv("./playlistData/spotify_dataset.csv", on_bad_lines='skip')

In [3]:
def printAmmount(start, ammount):
    for i in range(start, ammount + start):
        print(dfPlaylists.iloc[i])
        
printAmmount(500, 1)

user_id            07f0fc3be95dcd878966b1f9572ff670
 "artistname"                           Passion Pit
 "trackname"                             Sleepyhead
 "playlistname"                             Electro
Name: 500, dtype: object


In [4]:
playlists = []
playlistNames = []
dfPlaylists.iloc[0][3]
for i in range(100000): # change to the whole thing at end
    currentPLName = dfPlaylists.iloc[i][3]
    if (currentPLName not in playlistNames):
        playlistNames.append(currentPLName)
        playlists.append([])
        
    playlists[playlistNames.index(currentPLName)].append((dfPlaylists.iloc[i][1], dfPlaylists.iloc[i][2]))

In [5]:
print(len(playlists[0]))
print(len(playlistNames))

67
1904


In [6]:
data = {
    'grant_type': 'client_credentials',
    'client_id': 'ec3bd517542a46a7bb2d2af61972f4dc',
    'client_secret': 'ba508a2a2f9d49c1bfb04576085c5dc1'
}

accessToken = requests.post('https://accounts.spotify.com/api/token', data=data).json()

tokenHeader = {
    'Authorization': accessToken['token_type'] + ' ' + accessToken['access_token']
}

In [7]:
print(accessToken)

{'access_token': 'BQC9ZYeOJYGAlwpffBiDefnsZ5NIbe28vuhwn7kMUQhDVkvk8118iHBaMeGFA-jUzO0iIdF9lTqXPvp0wi3NgeH0-kU25g69_CxybFNCUPi4QEnLcVk', 'token_type': 'Bearer', 'expires_in': 3600}


In [8]:
def removeLastWord(trackName):
    words = []
    previous = 0
    new = 0
    for i in range(len(trackName)):
        foundSpace = False
        if (trackName[i] == ' '):
            foundSpace = True
            previous = new
            
        if (foundSpace):
            new = i
            words.append(trackName[previous : new])
            foundSpace = False
            
    newTrackName = ""
    for i in words:
        newTrackName += i
        
    return newTrackName

def removeFirstWord(trackName):
    words = []
    previous = 0
    new = 0
    for i in range(len(trackName)):
        foundSpace = False
        if (trackName[i] == ' '):
            foundSpace = True
            previous = new
            
        if (foundSpace):
            new = i
            words.append(trackName[previous : new])
            foundSpace = False
            
        if (i + 1 == len(trackName)):
            previous = new
            new = i + 1
            words.append(trackName[previous : new])
            
    newTrackName = ""
    for i in range(1, len(words)):
        newTrackName += words[i]
        
    if (len(newTrackName) > 0 and newTrackName[0] == " "):
        newTrackName = newTrackName[1:]
    return newTrackName

x1 = removeLastWord("Blackbird - Live at CitiField, NYC - Digital Audio")
x2 = removeFirstWord("Blackbird - Live at CitiField, NYC - Digital Audio")
#x3 = removeFirstWord(x2)
print(x2)

- Live at CitiField, NYC - Digital Audio


In [9]:
def getSpotifyIdByTrackName(trackName, trackArtist):
    params = {
        'q': 'track:' + trackName + " artist:" + trackArtist,
        'type': 'track',
        'limit': '1'
    }

    response = requests.get('https://api.spotify.com/v1/search', params=params, headers=tokenHeader).json()
    trackNameCopy = trackName
    
    while (len(response['tracks']['items']) <= 0 and len(trackName) > 0): # if it dosen't find a song, it removes the last word and tries again
        trackName = removeLastWord(trackName)
        params = {
            'q': 'track:' + trackName + " artist:" + trackArtist,
            'type': 'track',
            'limit': '1'
        }
        response = requests.get('https://api.spotify.com/v1/search', params=params, headers=tokenHeader).json()
    while (len(response['tracks']['items']) <= 0 and len(trackNameCopy) > 0):
        trackNameCopy = removeFirstWord(trackNameCopy)
        params = {
            'q': 'track:' + trackNameCopy + " artist:" + trackArtist,
            'type': 'track',
            'limit': '1'
        }
        response = requests.get('https://api.spotify.com/v1/search', params=params, headers=tokenHeader).json()
        
    if (len(trackNameCopy) <= 0):
        return ""
        
    return response['tracks']['items'][0]['id']

In [10]:
euphoriaID = getSpotifyIdByTrackName("euphoria", "Kendrick Lamar")
euphoriaCall = 'https://api.spotify.com/v1/tracks/' + euphoriaID
meetTheGrahmsID = getSpotifyIdByTrackName("meet the grahams", "Kendrick Lamar")
meetTheGrahmsCall = 'https://api.spotify.com/v1/tracks/' + meetTheGrahmsID
notLikeUsID = getSpotifyIdByTrackName("Not Like Us", "Kendrick Lamar")
notLikeUsCall = 'https://api.spotify.com/v1/tracks/' + notLikeUsID

pushUpsID = getSpotifyIdByTrackName("Push Ups", "Drake")
pushUpsCall = 'https://api.spotify.com/v1/tracks/' + pushUpsID
familyMattersID = getSpotifyIdByTrackName("Family Matters", "Drake")
familyMattersCall = 'https://api.spotify.com/v1/tracks/' + familyMattersID

euphoriaResponse = requests.get(euphoriaCall, headers=tokenHeader)
meetTheGrahmsResponse = requests.get(meetTheGrahmsCall, headers=tokenHeader)
notLikeUsResponse = requests.get(notLikeUsCall, headers=tokenHeader)
pushUpsResponse = requests.get(pushUpsCall, headers=tokenHeader)
familyMattersResponse = requests.get(familyMattersCall, headers=tokenHeader)

testID = getSpotifyIdByTrackName("Feeling Good", "Michael Bublé")
testCall = 'https://api.spotify.com/v1/tracks/' + testID
testResponse = requests.get(testCall, headers=tokenHeader)

In [11]:
print("(Kendrick) Euphoria popularity: " ,euphoriaResponse.json()['popularity'])
print("(Kendrick) meet the grahams popularity: " ,meetTheGrahmsResponse.json()['popularity'])
print("(Kendrick) not like us popularity: " ,notLikeUsResponse.json()['popularity'])
print("(Drake) push ups popularity: " ,pushUpsResponse.json()['popularity'])
print("(Drake) family matters popularity: " ,familyMattersResponse.json()['popularity'])
print("(Test) test popularity: " ,testResponse.json()['popularity'])

(Kendrick) Euphoria popularity:  90
(Kendrick) meet the grahams popularity:  79
(Kendrick) not like us popularity:  85
(Drake) push ups popularity:  90
(Drake) family matters popularity:  79
(Test) test popularity:  68


In [12]:
# test to make sure the id retrieved matches the song, dont run for every track to limit api calls
trackResponse = getSpotifyIdByTrackName("7 Years Too Late", "Tiffany Page")
if (len(trackResponse) > 0):
    resID = trackResponse
else:
    resID = ""
print("Search: ", "Years Too Late")
print("Result ID: ", resID)

Search:  Years Too Late
Result ID:  2bfDYHwwNgFETaikWy8iqX


In [13]:
# create playlistsById from playlists
# idealy do this with threading because its really slow
trackIDs = []
shape = dfPlaylists.shape
doPartial = len(playlists[0]) + len(playlists[1])

for i in range(doPartial):
    ID = getSpotifyIdByTrackName(dfPlaylists.iloc[i][2], dfPlaylists.iloc[i][1])
    trackIDs.append(ID)


In [14]:
for i in range(shape[0] - doPartial):
    trackIDs.append("")

In [15]:
if ("track_id" in dfPlaylists.columns):
    dfPlaylists = dfPlaylists.drop(columns = "track_id")
dfPlaylists.insert(4, "track_id", trackIDs)

dfPlaylists.head()
        

,user_id,"""artistname""","""trackname""","""playlistname""",track_id
0,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,(The Angels Wanna Wear My) Red Shoes,HARD ROCK 2010,0GmLrYUBXDC5vti77zBZfJ
1,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,"(What's So Funny 'Bout) Peace, Love And Unders...",HARD ROCK 2010,77XzsYwTkvLoveW01Lanrk
2,9cc0cfd4d7d7885102480dd99e7a90d6,Tiffany Page,7 Years Too Late,HARD ROCK 2010,2bfDYHwwNgFETaikWy8iqX
3,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello & The Attractions,Accidents Will Happen,HARD ROCK 2010,2Rv3hGdKSLeX30V55asE3s
4,9cc0cfd4d7d7885102480dd99e7a90d6,Elvis Costello,Alison,HARD ROCK 2010,1v98rfd0an913AzHvMNG8a


In [16]:
# dfPlaylists.to_csv("./playlistData/playlistDataWithID.csv")